In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

In [6]:
df = pd.read_csv("/content/creditcard.csv")           # Load the dataset to a dataframe

In [ ]:
df_train = df[:100000]                                            # We cut in two the original dataset
df_train_1 = df_train[df_train['Class'] == 1]                     # Separate the fraud data from the dataset
df_train_0 = df_train[df_train['Class'] == 0]                     # Separate the non-fraud dataset
print('Number of Fraud Transaction = ' + str(len(df_train_1)))    # Print the number of fraud data

df_sample=df_train_0.sample(len(df_train_1))                      # Taking the same number of non fraud samples as of fraud samples
df_train = pd.concat([df_train_1, df_sample], ignore_index=True)

df_shuffled = df_train.sample(frac=1.0, random_state=42)          # shuffle the training data
df_train.isnull().values.any()                                    # checkin if there is any NULL value is data
df_shuffled.head()

In [ ]:
# Split the data into features and labels
x = df_shuffled.drop('Class', axis=1)                             # Separate the input features i.e. 30 cols
y = df_shuffled['Class']                                          # Take the clas labels
x.shape

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train.shape

In [ ]:
# Standardize the features (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# fitting the model for the linear kernel

acc = []
acc_tr = []
coefficient = []
i=0
df = pd.DataFrame(columns = ['c','train_acc','test_acc'])
for c in [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]:
    svm = SVC(kernel='linear', C=c)
    svm.fit(X_train, y_train)                                 # fit the model with train data
    coef = svm.coef_                                          # taking coefficient(weights assigned to each feature) from the model

    p_tr = svm.predict(X_train)                               # y cap for the trian data
    a_tr = accuracy_score(y_train, p_tr)                      # accuracy for the predicted y cap value

    pred = svm.predict(X_test)                                # prediction for the test data
    a = accuracy_score(y_test, pred)                          # accuracy for the prediction on test data

    coefficient.append(coef)                                  # storing coefficient values for plotting
    acc_tr.append(a_tr)                                       # storing accuracy of the train data for plotting
    acc.append(a)                                             # storing accuracy of the test data for plotting

    df.loc[i] = [c,a_tr,a]                                    # storing cost parameter, train accuracy and test accuracy
    i=i+1

In [ ]:
df

In [ ]:
# plotting the testing and train accuracy for the different cose parameter c
c = [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]

plt.subplots(figsize=(10, 5))
plt.semilogx(c, acc,'-gD' ,color='red' , label="Testing Accuracy")
plt.semilogx(c, acc_tr,'-gD' , label="Training Accuracy")
plt.grid(True)
plt.xlabel("Cost Parameter C")
plt.ylabel("Accuracy")
plt.legend()
plt.title('Accuracy versus the Cost Parameter C (log-scale)')
plt.show()

In [ ]:
# fititng the model for the polynomial kernel
coefficient = []
n_supp = []
sup_vec = []
i = 0
df = pd.DataFrame(columns = ['c','degree','train_acc','test_acc'])
for c in [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]:
    for degree in [2,3,4,5,10]:
        svm = SVC(kernel='poly', C=c, degree=degree)
        model = svm.fit(X_train, y_train)
        globals()['model%s' % i] = model
        d_coef = svm.dual_coef_                                       # represents the coefficients
        support = svm.n_support_                                      # returns the no of support vectors for each class
        sv = svm.support_                                             # returns the indices of the support vectors

        p_tr = svm.predict(X_train)
        a_tr = accuracy_score(y_train, p_tr)

        pred = svm.predict(X_test)
        a = accuracy_score(y_test, pred)

        coefficient.append(d_coef)
        n_supp.append(support)
        sup_vec.append(sv)
        df.loc[i] = [c,degree,a_tr,a]
        i=i+1

In [ ]:
df

In [ ]:
# similarly fit the model with the RBF kernel

coefficient = []
n_supp = []
sup_vec = []
i = 0
df = pd.DataFrame(columns = ['c','gamma','train_acc','test_acc'])
for c in [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]:
    for g in [0.001, 0.01, 0.1, 1, 10, 100]:
        svm = SVC(kernel='rbf', C=c, gamma=g)
        model = svm.fit(X_train, y_train)
        globals()['model%s' % i] = model
        d_coef = svm.dual_coef_
        support = svm.n_support_
        sv = svm.support_

        p_tr = svm.predict(X_train)
        a_tr = accuracy_score(y_train, p_tr)

        pred = svm.predict(X_test)
        a = accuracy_score(y_test, pred)

        coefficient.append(d_coef)
        n_supp.append(support)
        sup_vec.append(sv)
        df.loc[i] = [c,g,a_tr,a]
        i=i+1

In [ ]:
df

In [ ]:
pd.DataFrame(coefficient[15]) # dual_coef_

In [ ]:
pd.DataFrame(coefficient[15]) # dual_coef_

The support vectors identified by the SVC each belong to a certain class (0 and
1). In the dual coefficients, they are ordered according to the class they belong to. There are as many rows in each group as there are support vectors i.e. 129.

In [ ]:
pd.DataFrame(n_supp[15]) # n_support_

,0
0,68
1,61


"nsupport" divides the number of support vestors by the class. So we can say that when class 0 has 68 support vectors, it means 68 are the positive support vectors and rest all are the negative support vectors for 0-versus-rest classifier.